In [1]:
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from hit_rate import HitRate
import pandas as pd

In [2]:
spark = SparkSession.builder.master("local[*]") \
                    .config('spark.ui.showConsoleProgress', 'false')\
                    .appName('MovieRecomender') \
                    .getOrCreate()
print(spark.sparkContext)
print("Spark App Name : "+ spark.sparkContext.appName)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 11:57:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<SparkContext master=local[*] appName=MovieRecomender>
Spark App Name : MovieRecomender


In [3]:
import os 
os.chdir ('/home/mt/MovieRecomender')
#change to your working directory
%pwd

'/home/mt/MovieRecomender'

In [4]:
df_m = pd.read_csv("data/movies.dat", engine='python', sep='::', names=["MovieID", "Title", "Genres"],encoding='ISO-8859-1')
df_m = spark.createDataFrame(df_m)

df_r = pd.read_csv("data/ratings.dat", engine='python', sep='::', names=["UserID", "MovieID", "Rating", "Timestamp"])
df_r = spark.createDataFrame(df_r)

df_u = pd.read_csv("data/users.dat", engine='python', sep='::', names=["UserID", "Gender", "Age", "Occupation", "Zip-code"])
df_u = spark.createDataFrame(df_u)

In [5]:
df_m.printSchema()
df_m.show(3)

root
 |-- MovieID: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genres: string (nullable = true)

+-------+--------------------+--------------------+
|MovieID|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
+-------+--------------------+--------------------+
only showing top 3 rows



In [6]:
df_r.printSchema()
df_r.show(3)

root
 |-- UserID: long (nullable = true)
 |-- MovieID: long (nullable = true)
 |-- Rating: long (nullable = true)
 |-- Timestamp: long (nullable = true)

22/10/05 11:57:40 WARN TaskSetManager: Stage 1 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
+------+-------+------+---------+
only showing top 3 rows



In [7]:
#df_u.printSchema()
#df_u.show(3)

root
 |-- UserID: long (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: long (nullable = true)
 |-- Occupation: long (nullable = true)
 |-- Zip-code: string (nullable = true)

+------+------+---+----------+--------+
|UserID|Gender|Age|Occupation|Zip-code|
+------+------+---+----------+--------+
|     1|     F|  1|        10|   48067|
|     2|     M| 56|        16|   70072|
|     3|     M| 25|        15|   55117|
+------+------+---+----------+--------+
only showing top 3 rows



In [8]:
df=df_m.join(df_r,"MovieID","inner")#.join(df_u,"UserID","inner")
df.printSchema()
df.show(3)

root
 |-- MovieID: long (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- UserID: long (nullable = true)
 |-- Rating: long (nullable = true)
 |-- Timestamp: long (nullable = true)

22/10/05 11:57:41 WARN TaskSetManager: Stage 4 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
+-------+--------------------+--------------------+------+------+---------+
|MovieID|               Title|              Genres|UserID|Rating|Timestamp|
+-------+--------------------+--------------------+------+------+---------+
|     29|City of Lost Chil...|    Adventure|Sci-Fi|     5|     5|978245065|
|   1806|       Paulie (1998)|Adventure|Childre...|     6|     3|978236876|
|    474|In the Line of Fi...|     Action|Thriller|     7|     5|978234842|
+-------+--------------------+--------------------+------+------+---------+
only showing top 3 rows



In [9]:
df=df.select(["UserID","MovieID","Rating"])
df.na.drop()
df.show(3)

22/10/05 11:57:43 WARN TaskSetManager: Stage 10 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
+------+-------+------+
|UserID|MovieID|Rating|
+------+-------+------+
|     5|     29|     5|
|     6|   1806|     3|
|     7|    474|     5|
+------+-------+------+
only showing top 3 rows



In [10]:
(train, test) = df.randomSplit([0.8, 0.2])

In [11]:
als = ALS(userCol="UserID", itemCol="MovieID", ratingCol="Rating", nonnegative = True, implicitPrefs = False)

In [12]:
grid_search = ParamGridBuilder().addGrid(als.rank,[10]).addGrid(als.maxIter,[10]).addGrid(als.regParam, [0.1] ).build()

In [13]:
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating", predictionCol="prediction") 

In [14]:
cv = CrossValidator(estimator=als, estimatorParamMaps=grid_search, evaluator=evaluator, numFolds=5)

In [15]:
spark.sparkContext.setCheckpointDir('checkpoint/')
cv_fitted=cv.fit(train)

22/10/05 11:57:45 WARN TaskSetManager: Stage 16 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 11:57:56 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/10/05 11:57:56 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/10/05 11:58:01 WARN TaskSetManager: Stage 92 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 11:58:06 WARN TaskSetManager: Stage 153 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 11:58:13 WARN TaskSetManager: Stage 228 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 11:58:16 WARN TaskSetManager: Stage 294 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 11:58:22 WARN TaskSetManager: St

In [16]:
print(cv_fitted.bestModel.rank, cv_fitted.bestModel._java_obj.parent().getMaxIter(),cv_fitted.bestModel._java_obj.parent().getRegParam())

10 10 0.1


In [17]:
evaluator.evaluate(cv_fitted.transform(test).na.drop())

22/10/05 11:58:51 WARN TaskSetManager: Stage 798 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.


0.8714815538129977

In [18]:
hr_evaluator = HitRate(predictionCol='prediction', labelCol='rating', userCol='userId')
model = als.fit(train)
predictions = model.transform(test)
hr_evaluator.evaluate(predictions)

22/10/05 11:58:53 WARN TaskSetManager: Stage 859 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 11:58:58 WARN TaskSetManager: Stage 932 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
+------+-----+
|userId|count|
+------+-----+
|     1|    9|
|     5|   10|
|     6|   10|
|     7|    6|
|     9|   10|
|    10|   10|
|    17|   10|
|    19|   10|
|    22|   10|
|    25|   10|
|    26|   10|
|    27|   10|
|    28|   10|
|    29|   10|
|    31|   10|
|    32|    7|
|    33|   10|
|    34|   10|
|    39|   10|
|    41|    7|
+------+-----+
only showing top 20 rows

22/10/05 11:59:00 WARN TaskSetManager: Stage 997 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.
22/10/05 11:59:02 WARN TaskSetManager: Stage 1062 contains a task of very large size (3861 KiB). The maximum recommended task size is 1000 KiB.


1.0